In [1]:
import pandas as pd

In [2]:
trips_df = pd.read_pickle('../00_data/trips.pkl')

In [3]:
trips_df['start_time_floored'] = trips_df['start_time'].dt.floor('1H')
trips_df['end_time_floored'] = trips_df['end_time'].dt.floor('1H')

In [4]:
starting_trips_grouped = (
    trips_df.groupby(["start_time_floored"])
    .size()
    .to_frame("starting_trips")
)
ending_trips_grouped = (
    trips_df.groupby(["end_time_floored"])
    .size()
    .to_frame("ending_trips")
)

In [5]:
trips_hourly = pd.concat([starting_trips_grouped, ending_trips_grouped], axis=1)
trips_hourly.isna().sum()

starting_trips    142
ending_trips       85
dtype: int64

In [6]:
# some missing values exits because there may be hours at which trips start but
# no trips end and vice versa
trips_hourly = trips_hourly.fillna(0) 

In [7]:
trips_hourly.to_pickle('../00_data/trips_hourly.pkl')